In [1]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


In [2]:
def refund_flight (flight_PNR:str) ->str:
    return f"Refunded Flight with PNR {flight_PNR}"

In [ ]:
api_key=None
model_client = OpenAIChatCompletionClient(model='gpt-4o',api_key=api_key,parallel_tool_calls=False)

In [14]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

In [ ]:
flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight PNR numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [ ]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)


In [ ]:
await team.reset()

In [ ]:
# if (isinstance(message, HandoffMessage))

In [22]:
task = ' I want to refund my flight'

async def run_team_stream() -> None:

    task_result = await Console(team.run_stream(task = task))

    last_message = task_result.messages[-1]


    while ( isinstance(last_message,HandoffMessage) and last_message.target == 'user'):

        user_ka_message = input("User : ")

        task_result = await Console(team.run_stream(task = HandoffMessage(source='user',target=last_message.source,content=user_ka_message)))

        last_message = task_result.messages[-1]

await run_team_stream()

---------- TextMessage (user) ----------
 I want to refund my flight


---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_HAIEWNBnUBHvnlYeSwpEFvWK', arguments='{}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_HAIEWNBnUBHvnlYeSwpEFvWK', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
[FunctionCall(id='call_HAIEWNBnUBHvnlYeSwpEFvWK', arguments='{}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_HAIEWNBnUBHvnlYeSwpEFvWK', is_error=False)]
---------- HandoffMessage (travel_agent) -------

In [23]:
flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight PNR numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [24]:
await team.reset()

In [ ]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE") 
team = Swarm([flights_refunder,travel_agent ], termination_condition=termination,max_turns=20)


In [ ]:
task = ' I want to refund my flight'

async def run_team_stream() -> None:

    task_result = await Console(team.run_stream(task = task))

    last_message = task_result.messages[-1]


    while ( isinstance(last_message,HandoffMessage) and last_message.target == 'user'):

        user_ka_message = input("User : ")

        task_result = await Console(team.run_stream(task = HandoffMessage(source='user',target=last_message.source,content=user_ka_message)))

        last_message = task_result.messages[-1]

await run_team_stream()

---------- TextMessage (user) ----------
 I want to refund my flight
 I want to refund my flight
---------- TextMessage (flights_refunder) ----------
To process a refund for your flight, I will need the flight PNR (Passenger Name Record) number. Could you please provide that information?
---------- TextMessage (flights_refunder) ----------
To process a refund for your flight, I will need the flight PNR (Passenger Name Record) number. Could you please provide that information?
---------- TextMessage (flights_refunder) ----------
To proceed with refunding your flight, I'll need your flight PNR (Passenger Name Record) number. Could you please provide that to me?
---------- TextMessage (flights_refunder) ----------
To proceed with refunding your flight, I'll need your flight PNR (Passenger Name Record) number. Could you please provide that information?
---------- TextMessage (flights_refunder) ----------
To process a refund for your flight, I will need the flight PNR (Passenger Name Record